## PyTorch: New Autograd Functions

In [1]:
import torch
from torch.autograd import Variable

In [3]:
# Define your own autograd functions by writing forward and backward for Tensors
class ReLU(torch.autograd.Function):
    def forward(self, x):
        self.save_for_backward(x)
        return x.clamp(min=0)
    def backward(self, grad_y):
        x, = self.saved_tensors
        grad_input = grad_y.clone()
        grad_input[x < 0] = 0
        return grad_input

In [4]:
N, D_in, H, D_out = 64, 1000, 100, 10

x = Variable(torch.randn(N, D_in), requires_grad=False)
y = Variable(torch.randn(N, D_out), requires_grad=False)
w1 = Variable(torch.randn(D_in, H), requires_grad=True)
w2 = Variable(torch.randn(H, D_out), requires_grad=True)

In [7]:
learning_rate = 1e-6
for t in range(500):
    # Can use our new autograd function in the forward pass
    relu = ReLU()
    y_pred = relu(x.mm(w1)).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    
    # Compute gradients of loss with respect to w1 and w2(zero out grads first)
    if hasattr(w1, 'grad'): w1.grad.data.zero_()
    if hasattr(w2, 'grad'): w2.grad.data.zero_()
    loss.backward()
    
    # Make gradient descent step on weights
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data